In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import scale, StandardScaler
import warnings
warnings.filterwarnings("ignore")

In [7]:
df = pd.read_csv("Hitters.csv")
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [8]:
df.dropna(inplace=True)

df = pd.get_dummies(df, columns=["League","Division","NewLeague"])

df.drop(["League_N", "Division_W", "NewLeague_N"], axis=1, inplace=True)

df.replace(to_replace=False, value=0, inplace=True)
df.replace(to_replace=True, value=1, inplace=True)

df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Salary,League_A,Division_E,NewLeague_A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,632,43,10,475.0,0,0,0
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,880,82,14,480.0,1,0,1
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,200,11,3,500.0,0,1,0
4,321,87,10,39,42,30,2,396,101,12,48,46,33,805,40,4,91.5,0,1,0
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,282,421,25,750.0,1,0,1


In [9]:
X = df.drop("Salary", axis=1)
y = df[["Salary"]]
X.shape, y.shape

((263, 19), (263, 1))

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=16)

In [21]:
# Yapay sinir ağları homojen verilerde daha iyi performans gösterdiği için 
# eğitim ve test verilerini dönüştürüyoruz

In [11]:
scaler = StandardScaler()

In [12]:
X_train_scaled = scaler.fit_transform(X_train)

In [18]:
X_test_scaled = scaler.fit_transform(X_test)

In [31]:
mlp_model = MLPRegressor(verbose=True).fit(X_train_scaled, y_train)

Iteration 1, loss = 238392.85100084
Iteration 2, loss = 238351.56573495
Iteration 3, loss = 238310.17853441
Iteration 4, loss = 238268.82758805
Iteration 5, loss = 238227.44016465
Iteration 6, loss = 238186.25553687
Iteration 7, loss = 238145.08161172
Iteration 8, loss = 238103.86019879
Iteration 9, loss = 238062.65719360
Iteration 10, loss = 238021.37560692
Iteration 11, loss = 237980.14217231
Iteration 12, loss = 237938.89384740
Iteration 13, loss = 237897.37752713
Iteration 14, loss = 237855.57249728
Iteration 15, loss = 237813.49042072
Iteration 16, loss = 237771.26737842
Iteration 17, loss = 237728.98487271
Iteration 18, loss = 237686.59464247
Iteration 19, loss = 237644.03716854
Iteration 20, loss = 237601.21288534
Iteration 21, loss = 237558.18742474
Iteration 22, loss = 237514.90112953
Iteration 23, loss = 237471.33899481
Iteration 24, loss = 237427.54422915
Iteration 25, loss = 237383.52936349
Iteration 26, loss = 237339.24261541
Iteration 27, loss = 237294.69449288
Iteration 

In [32]:
mlp_model.get_params()

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 200,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': True,
 'warm_start': False}

In [33]:
y_pred = mlp_model.predict(X_test_scaled)

In [35]:
# rmse
np.sqrt(mean_squared_error(y_test, y_pred))

688.9997042347404

In [36]:
# Model Tuning

In [82]:
mlp_params = {
    "alpha": np.logspace(-3, 0, 4),
    "hidden_layer_sizes": [(20,5), (20,10,10)],
    "max_iter": [2500, 5000, 10000]
}

In [83]:
mlp_cv = GridSearchCV(MLPRegressor(), mlp_params, cv=5, n_jobs=-1, verbose=2).fit(X_train_scaled, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


In [84]:
mlp_cv.best_params_

{'alpha': 0.001, 'hidden_layer_sizes': (20, 5), 'max_iter': 2500}

In [52]:
# Final Model

In [88]:
mlp_tuned = MLPRegressor(alpha=0.001, hidden_layer_sizes=(20,5), max_iter=2500 ,verbose=True).fit(X_train_scaled, y_train)
#alpha: L2 normu, hiden_layer_sizes: katman ve nöron sayıları, max_iter: epoch(döngü) sayısı 

Iteration 1, loss = 238789.18935087
Iteration 2, loss = 238774.31001730
Iteration 3, loss = 238759.54425763
Iteration 4, loss = 238744.84717500
Iteration 5, loss = 238730.23681543
Iteration 6, loss = 238715.74327989
Iteration 7, loss = 238701.27519974
Iteration 8, loss = 238686.82610622
Iteration 9, loss = 238672.62672319
Iteration 10, loss = 238658.45658563
Iteration 11, loss = 238644.11149784
Iteration 12, loss = 238629.74619902
Iteration 13, loss = 238615.39075606
Iteration 14, loss = 238601.04154005
Iteration 15, loss = 238586.94904025
Iteration 16, loss = 238572.95019826
Iteration 17, loss = 238559.05332644
Iteration 18, loss = 238545.25432341
Iteration 19, loss = 238531.42302875
Iteration 20, loss = 238517.59749711
Iteration 21, loss = 238503.82184220
Iteration 22, loss = 238490.04818647
Iteration 23, loss = 238475.87571180
Iteration 24, loss = 238461.72152915
Iteration 25, loss = 238447.58122980
Iteration 26, loss = 238433.63754083
Iteration 27, loss = 238419.77823832
Iteration 

In [89]:
y_pred = mlp_tuned.predict(X_test_scaled)

In [90]:
np.sqrt(mean_squared_error(y_test, y_pred))

234.72586292013628